In [1]:
import pandas as pd
import json
drider = pd.read_csv(r'.\data\rider_infos.csv')

drace = pd.read_csv(r'.\data\results\Results2017CatWT.csv')


In [2]:
# gather previous results
from os import listdir
from os.path import isfile, join, split
from pathlib import Path

mypath = r".\data\results"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# data van 2012 t/m 2021 --> neem 3 jaar in het verleden  mee, dus van 2016 t/m 2021 --> index 3 t/m 8


Dit is code om de CSV's naar SQLite te converteren, verder niet van belang... De studenten krijgen de SQL

In [3]:
import pandas as pd
import hashlib
import sqlite3
import pandas as pd
import os

# Define database file name
db_file = "cycling_big.db"

# Remove the existing database file if it exists
if os.path.exists(db_file):
    os.remove(db_file)

# Connect to SQLite database (this will create a new, clean file)
conn = sqlite3.connect(db_file)
# Enable foreign key support
conn.execute("PRAGMA foreign_keys = ON;")

# Step 1: Create tables with the correct schema and foreign key constraints
conn.execute("""
CREATE TABLE riders (
    rider_id TEXT PRIMARY KEY,
    fullname TEXT NOT NULL,
    team TEXT,
    birthdate TEXT NOT NULL,
    country TEXT,
    height REAL,
    weight REAL,
    rider_url TEXT,
    pps TEXT,
    rdr TEXT
);
""")

conn.execute("""
CREATE TABLE race_results (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Rnk TEXT,
    GC REAL,
    Timelag TEXT,
    BiB TEXT,
    Rider TEXT,
    Age INTEGER,
    Team TEXT,
    UCI REAL,
    Pnt REAL,
    Time TEXT,
    Circuit INTEGER,
    Race_Name TEXT,
    Stage_Name TEXT,
    Date TEXT,
    Stage_Type TEXT,
    Start TEXT,
    Finish TEXT,
    Race_ID INTEGER,
    Stage_Number INTEGER,
    Length TEXT,
    Category TEXT,
    Race_url TEXT,
    Stage_url TEXT,
    rider_id TEXT,
    FOREIGN KEY (rider_id) REFERENCES riders(rider_id) ON DELETE SET NULL
);
""")

# Load your datasets
riders = pd.read_csv(r'.\data\rider_infos.csv')
# Step 1: Create a unique rider_id in the riders dataset
def generate_rider_id(row):
    unique_string = f"{row['fullname']}_{row['birthdate']}"
    return hashlib.md5(unique_string.encode()).hexdigest()

riders['rider_id'] = riders.apply(generate_rider_id, axis=1)
riders = riders.drop(columns=['Unnamed: 0'])
riders.to_sql("riders", conn, if_exists="append", index=False)

# race_results = pd.read_csv(r'.\data\results\Results2017CatWT.csv')
# print(riders.columns)
# print(riders.dtypes)

# print(race_results.columns)
# print(race_results.dtypes)

for i in range(len(onlyfiles)): #range(3,9):
    f = r'.\data\results\{}'.format(onlyfiles[i])
    print(f)
    race_results = pd.read_csv(f)
    # Step 2: Map rider_id to the race results dataset
    # Ensure 'Rider' in race_results matches 'fullname' in riders
    race_results = race_results.merge(
        riders[['fullname', 'rider_id']],
        left_on='Rider',
        right_on='fullname',
        how='left'
    )
    # Remove the redundant 'fullname' column added during the merge
    race_results.drop(columns=['fullname'], inplace=True)

    # Step 2: Insert data into the tables
    race_results = race_results.drop(columns=['Unnamed: 0'])
    race_results.rename(columns={'Stage#': 'Stage_Number'}, inplace=True)
    # Write to SQL

    race_results.to_sql("race_results", conn, if_exists="append", index=False)

# Close the connection
conn.close()

.\data\results\Results2012CatWT.csv
.\data\results\Results2014CatWT.csv
.\data\results\Results2015CatWT.csv
.\data\results\Results2016CatWT.csv
.\data\results\Results2017CatWT.csv
.\data\results\Results2018CatWT.csv
.\data\results\Results2019CatWT.csv
.\data\results\Results2020CatWT.csv
.\data\results\Results2021CatWT.csv


In [4]:
conn.close()

Dit is code iom de velder pps en rdr die json achtige subdata bevatten om te zetten naar nieuwe colommen. Dit wordt nog niet teogepast hieronder voor de ML dataset... TODO i guess

In [5]:
tojson = lambda x: json.loads(x)
replace = lambda x: x.replace("'","\"")

pps = drider["pps"].apply(replace)
arecords = pps.apply(tojson).to_list()
a = pd.DataFrame.from_dict(arecords)
dnew = pd.concat((drider,a),axis=1)

rdr = drider["rdr"].apply(replace)
arecords = rdr.apply(tojson).to_list()
a = pd.DataFrame.from_dict(arecords)
dnew = pd.concat((dnew,a),axis=1)


d_all = drace.merge(right=dnew, left_on="Rider", right_on="fullname")
print(d_all.columns)
d_all.drop(columns=['Unnamed: 0_x', 'Time', 'BiB', 
       'Start', 'Finish', 'Race_ID', 'Stage#', 'Race_url', 
       'Stage_url', 'Unnamed: 0_y', 'rider_url', 'pps', 'rdr'],inplace=True)

Index(['Unnamed: 0_x', 'Rnk', 'GC_x', 'Timelag', 'BiB', 'Rider', 'Age', 'Team',
       'UCI', 'Pnt', 'Time', 'Circuit', 'Race_Name', 'Stage_Name', 'Date',
       'Stage_Type', 'Start', 'Finish', 'Race_ID', 'Stage#', 'Length',
       'Category', 'Race_url', 'Stage_url', 'Unnamed: 0_y', 'fullname', 'team',
       'birthdate', 'country', 'height', 'weight', 'rider_url', 'pps', 'rdr',
       'One day races', 'GC_y', 'Time trial', 'Sprint', 'Climber',
       'PCS Ranking', 'UCI World Ranking', 'Specials | All Time Ranking'],
      dtype='object')


Dit is de code om de historische data te transformeren van een tabel met rider/etappe/race uitslagen, naar iets dat gaat over de plaatsing van een rider over de afgelopen n jaren. Deze maakt nu gebruikt van het veld "Pnt" in de originele data.

In [6]:
frames=[]
for i in range(3,9): #range(3,9):
    f = r'.\data\results\{}'.format(onlyfiles[i])
    print(f)
    drace = pd.read_csv(f)
    drace = drace.merge(right=dnew, left_on="Rider", right_on="fullname")
    #print(i)
    for j in range(1,4):
        idx = i-j
        f = r'.\data\results\{}'.format(onlyfiles[idx])
        print('   '+f)
        d3 = pd.read_csv(f)
        gr = d3.groupby(["Race_Name","Rider"])
        gr = gr["Pnt"].sum()
        race_rider = gr.unstack(level=0)
        race_rider["Sum"] = race_rider.sum(axis=1, skipna=True)
        
        totalpoints = race_rider.pop("Sum")
        
        drace = drace.merge(right=totalpoints, left_on="Rider", right_index=True)
        
        #col_name = Path(f).stem
        col_name = 'sumres_'+str(j)
        
        drace = drace.rename(columns = {"Sum" : col_name})
    #race_rider.head(15)
    frames.append(drace)

drace = pd.concat(frames)

.\data\results\Results2016CatWT.csv
   .\data\results\Results2015CatWT.csv
   .\data\results\Results2014CatWT.csv
   .\data\results\Results2012CatWT.csv
.\data\results\Results2017CatWT.csv
   .\data\results\Results2016CatWT.csv
   .\data\results\Results2015CatWT.csv
   .\data\results\Results2014CatWT.csv
.\data\results\Results2018CatWT.csv
   .\data\results\Results2017CatWT.csv
   .\data\results\Results2016CatWT.csv
   .\data\results\Results2015CatWT.csv
.\data\results\Results2019CatWT.csv
   .\data\results\Results2018CatWT.csv
   .\data\results\Results2017CatWT.csv
   .\data\results\Results2016CatWT.csv
.\data\results\Results2020CatWT.csv
   .\data\results\Results2019CatWT.csv
   .\data\results\Results2018CatWT.csv
   .\data\results\Results2017CatWT.csv
.\data\results\Results2021CatWT.csv
   .\data\results\Results2020CatWT.csv
   .\data\results\Results2019CatWT.csv
   .\data\results\Results2018CatWT.csv


Dit is code voor variable selectie en transformatie (category naar one-hot etc..)

In [7]:
import numpy as np

c = drace.columns
#print(c)
sel = ["Rnk", "GC_x", "Age", "Pnt", "Stage_Type", "Length", 'height', 'weight', 'One day races', 'GC_y', 'Time trial', 'Sprint', 'Climber',
       'PCS Ranking', 'UCI World Ranking', 'Specials | All Time Ranking',
       'sumres_1', 'sumres_2', 'sumres_3']

sel = ["Rnk", "GC_x", "Age", "Pnt", "Stage_Type", "Length", 'height', 'weight', 'One day races', 'GC_y', 'Time trial', 'Sprint', 'Climber',
       'PCS Ranking', 'UCI World Ranking', 'Specials | All Time Ranking',
       'sumres_1', 'sumres_2', 'sumres_3']

drace_sel = drace[sel]
#print(drace_sel.dtypes)
#print(drace_sel["Pnt"][1]+1)
pntsel = drace_sel["Pnt"].isna()
#drace_sel["Rnk"].replace(to_replace="DNF", value=np.NaN, inplace=True)
#drace_sel["Rnk"].replace(to_replace="OTL", value=np.NaN, inplace=True)
ss = drace_sel.Rnk.str.isnumeric()
drace_sel = drace_sel[drace_sel.Rnk.str.isnumeric()]

drace_sel.Stage_Type = drace_sel.Stage_Type.astype("category")
drace_sel.Length = drace_sel.Length.apply(lambda x: float(x.split(" ")[0]))
drace_sel = pd.get_dummies(drace_sel, drop_first=True, columns=["Stage_Type"])
drace_sel = drace_sel.astype(np.float16, copy=False)
print(drace_sel.dtypes)
#drace_sel=drace_sel.iloc[not pntsel.to_numpy(),:]
#drace_sel.dropna(inplace=True, subset=["Pnt"])

#drace_sel_na = drace_sel.dropna()
drace_sel = drace_sel.drop(columns=["Pnt"])
drace_sel["Rnk"] = drace_sel["Rnk"]<=30 #maak label binair (wel/geen 30 pnt)?
drace_sel["Rnk"] = drace_sel["Rnk"].astype(np.int8)

drace_sel.dropna(inplace=True)

y = drace_sel["Rnk"]

X = drace_sel.drop(columns=["Rnk"])


Rnk                            float16
GC_x                           float16
Age                            float16
Pnt                            float16
Length                         float16
height                         float16
weight                         float16
One day races                  float16
GC_y                           float16
Time trial                     float16
Sprint                         float16
Climber                        float16
PCS Ranking                    float16
UCI World Ranking              float16
Specials | All Time Ranking    float16
sumres_1                       float16
sumres_2                       float16
sumres_3                       float16
Stage_Type_RR                  float16
dtype: object


In [41]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

#model = LogisticRegression(solver="liblinear", verbose=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

pipe = make_pipeline(StandardScaler(), LogisticRegression(solver="lbfgs", verbose=0, fit_intercept=True, penalty="l2"))
pipe.fit(X_train, y_train)  # apply scaling on training data

print(pipe.score(X_test, y_test))

#model.fit(X_train, y_train)

yhat = pipe.predict(X_test)
phat = pipe.predict_proba(X_test)

acc =  accuracy_score(y_test, yhat)

cmat = confusion_matrix(y_test, yhat)

print(classification_report(y_test, yhat, target_names=["top x", "losers"]))

print(cmat)

0.7928513403736799
              precision    recall  f1-score   support

       top x       0.84      0.87      0.85      5935
      losers       0.68      0.62      0.65      2682

    accuracy                           0.79      8617
   macro avg       0.76      0.75      0.75      8617
weighted avg       0.79      0.79      0.79      8617

[[5159  776]
 [1009 1673]]


In [8]:
drace.dtypes
a= drace.Length[0]
b= float(a.split(" ")[0])